## Header 
Author : Amina Matt and Yichen Wang  
Date created : 14.10.2021  
Date last modified : 14.10.2021  
Python version : 3.8  
Description : Text processing of the CARICOM Compilation Archive (CCA) https://louverture.ch/cca/ 



In [12]:
# -*- coding: utf-8 -*-

import nltk
nltk.download('punkt')
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

PATH = '/Users/aminamatt/Documents/Cours-2021/FDH/Colonial-heritage-in-Switzerland/'

#Stanford NER 
NER_FOLDER = PATH+'NER-Standford/stanford-ner-2020-11-17'
CLASSIFIER_PATH = NER_FOLDER+'/classifiers/'
JAR_PATH = NER_FOLDER+'/stanford-ner.jar'

#classifiers
classifier_3 = 'english.all.3class.distsim.crf.ser.gz'#3 class model for recognizing locations, persons, and organizations
classifier_4 = 'english.conll.4class.distsim.crf.ser.gz'#4 class model for recognizing locations, persons, organizations, and miscellaneous entities
classifier_7 = 'english.muc.7class.distsim.crf.ser.gz' #7 class model for recognizing locations, persons, organizations, times, money, percents, and dates

st = StanfordNERTagger(CLASSIFIER_PATH+classifier_3, JAR_PATH, encoding='utf-8')

#Text retrieving
caricom_sample = PATH+'Caricom_Archive_Sample.txt'
caricom = PATH+'Caricom_Archive.txt'

text = open(caricom_sample, 'r').read()
tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

[nltk_data] Downloading package punkt to /Users/aminamatt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('=', 'O'), ('>', 'O'), ('Hans', 'PERSON'), ('Conrad', 'PERSON'), ('Hottinger', 'PERSON'), ('’', 'O'), ('s', 'O'), ('business', 'O'), ('partner', 'O'), ('was', 'O'), ('Denis', 'PERSON'), ('de', 'PERSON'), ('Rougemont', 'PERSON'), ('(', 'O'), ('1759\xad–1839', 'O'), (')', 'O'), ('from', 'O'), ('Saint-Aubin', 'LOCATION'), ('and', 'O'), ('Neuchâtel', 'LOCATION'), (',', 'O'), ('banker', 'O'), (',', 'O'), ('Prussian', 'O'), ('financial', 'O'), ('agent', 'O'), (',', 'O'), ('major', 'O'), ('Banque', 'ORGANIZATION'), ('de', 'ORGANIZATION'), ('France', 'ORGANIZATION'), ('shareholder', 'O'), ('and', 'O'), ('real', 'O'), ('estate', 'O'), ('owner', 'O'), ('in', 'O'), ('Paris', 'LOCATION'), ('and', 'O'), ('Berne', 'PERSON'), ('.', 'O'), ('He', 'O'), ('bought', 'O'), ('the', 'O'), ('Hôtel', 'O'), ('DuPeyrou', 'O'), ('in', 'O'), ('Neuchâtel', 'LOCATION'), ('in', 'O'), ('1816', 'O'), ('.', 'O'), ('In', 'O'), ('1837', 'O'), (',', 'O'), ('his', 'O'), ('son', 'O'), ('Abraham', 'PERSON'), ('Denis', 'PERS